In [1]:
import numpy as np
import os
import tempfile
import keras
from keras import backend as K
from keras import layers
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
from keras.callbacks import History 
from keras.models import load_model
from keras.layers import Embedding
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.layers import Concatenate
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import random
from termcolor import colored
print("import loads done")

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


import loads done


In [2]:
#data
# UPLOAD DATA
# (each user should put datafiles in this directory on their computer)
datapath = "blast_tab_1hit.out"
file = open(datapath, 'r')

same_entries = []
diff_entries = []
train_entries=[]
test_entries=[]
val_entries=[]
max_length_in = 0
max_length_out = 0
count_diff_entries_train=0
count_same_entries_train=0
count_diff_entries_val=0
count_same_entries_val=0
count_diff_entries_test=0
count_same_entries_test=0

np.random.seed(0)

for ln in file:
    toks = ln.split('\t')
    rand_num= np.random.random()
    
    max_length_in = max(max_length_in,len(toks[2]))
    max_length_out = max(max_length_out,len(toks[3]))
    if(toks[2] != toks[3]):
        if rand_num < 0.95:
            train_entries.append([toks[2], toks[3]])
            count_diff_entries_train += 1
        elif rand_num <0.975:
            test_entries.append([toks[2], toks[3]])
            count_diff_entries_test += 1
        else:
            val_entries.append([toks[2], toks[3]])
            count_diff_entries_val += 1
        
    if toks[2] == toks[3]:
        if rand_num > 0.975:
            val_entries.append([toks[2], toks[3]])
            count_same_entries_val += 1
        elif rand_num > 0.95:
            test_entries.append([toks[2], toks[3]])
            count_same_entries_test += 1
        elif rand_num > 0.9:
            train_entries.append([toks[2], toks[3]])
            count_same_entries_train += 1
        #same_entries.append([toks[2], toks[3]])
    #else:
        #diff_entries.append([toks[2], toks[3]])

file.close()
#num_entries = len(same_entries) + len(diff_entries)

#display train, test, val set
#print("train")
#print(train_entries[2])
#print("val")
#print(val_entries[2])
#print("test")
#print(test_entries[2])


#diff_entries_output[1]
one_hot_input = {'A': 0, 'T': 1, 'C': 2, 'G': 3, '-': 4}
one_hot_output = {'A': 0, 'T': 1, 'C': 2, 'G': 3}

In [3]:
#Checkpoint prints
#prints
print("train len",len(train_entries))
print("val len",len(val_entries))
print("test len",len(test_entries))
print("diff entries in train", count_diff_entries_train)
print("same entries train", count_same_entries_train)
print("diff entries in val", count_diff_entries_val)
print("same entries val", count_same_entries_val)
print("diff entries in test", count_diff_entries_test)
print("same entries test", count_same_entries_test)

print("total train + dev + test")
print(len(train_entries)+ len(val_entries)+ len(test_entries))

print("max len in",max_length_in)
print("max len out", max_length_out)

print("one train entry",train_entries[1])

train len 77510
val len 12269
test len 12107
diff entries in train 55906
same entries train 21604
diff entries in val 1450
same entries val 10819
diff entries in test 1507
same entries test 10600
total train + dev + test
101886
max len in 351
max len out 351
one train entry ['ACAGAGGGGGCAAGCGTTGTCCGGAGTTACTGGGCGTAAAGGGCGCGCAGGCGGTGGGCTGCGTCGGCGCTGAAAGCGCCCCGCTTAACGGGGCGAGGCGCGCCGATACGAGTCCACTCGAGGCAAGCAGAGGGTGGCGGAATTCCGGGTGGAGTGGTGAAATGCGTAGAGATCCGGAGGAACGCCGGTGGGGAAGCCGGCCACCTGGGCTTGACCTGACGCTGCGGCGCGACAGCGTGGGGAGCAAACCG', 'ACAGAGGGGGCAAGCGTTGTCCGGAGTTACTGGGCGTAAAGGGCGCGCAGGCGGTGGGCTGCGTCGGCGCTGAAAGCGCCCCGCTTAACGGGGCGAGGCGCGCCGATACGAGTCCACTCGAGGCAAGCAGAGGGTGGCGGAATTCCGGGTGGAGCGGTGAAATGCGTAGAGATCCGGAGGAACGCCGGTGGGGAAGCCGGCCACCTGGGCTTGACCTGACGCTGCGGCGCGACAGCGTGGGGAGCAAACCG']


In [4]:
input_seqs= [entry[0] for entry in train_entries]
output_seqs= [entry[1] for entry in train_entries]
val_input_seqs= [entry[0] for entry in val_entries]
val_output_seqs= [entry[1] for entry in val_entries]
test_input_seqs= [entry[0] for entry in test_entries]
test_output_seqs= [entry[1] for entry in test_entries]
#display
#print("train in")
#print(input_seqs[100])
#print("val in")
#print(val_input_seqs[100])
#print("test in")
#print(test_input_seqs[100])

In [5]:
print(len(input_seqs))
print(len(output_seqs))
print(len(val_input_seqs))
print(len(test_input_seqs))

77510
77510
12269
12107


In [6]:
labels_train= [[1]]* count_diff_entries_train + [[-1]]*count_same_entries_train
labels_val= [[1]]* count_diff_entries_val + [[-1]]*count_same_entries_val
labels_test= [[1]]* count_diff_entries_test + [[-1]]*count_same_entries_test
print("train labels len",len(labels_train))
print("val labels len",len(labels_val))
print("test labels len",len(labels_test))

train labels len 77510
val labels len 12269
test labels len 12107


In [7]:
c = list(zip(input_seqs, output_seqs, labels_train))
#seed = 123
#random.seed(seed)
#random.shuffle(c)
input_seqs, output_seqs, labels_train = zip(*c)
print("lengths:")
print("input_seqs->",len(input_seqs))
print("output_seqs ->", len(output_seqs))
print("labels train-->", len(labels_train))

lengths:
input_seqs-> 77510
output_seqs -> 77510
labels train--> 77510


In [8]:
print("lengths:")
print("input_seqs->",len(val_input_seqs))
print("output_seqs ->", len(val_output_seqs))
print("labels train-->", len(labels_val))

lengths:
input_seqs-> 12269
output_seqs -> 12269
labels train--> 12269


In [9]:
print("lengths:")
print("input_seqs->",len(test_input_seqs))
print("output_seqs ->", len(test_output_seqs))
print("labels train-->", len(labels_test))

lengths:
input_seqs-> 12107
output_seqs -> 12107
labels train--> 12107


In [10]:
#one hot encode train data

#ENCODE
encoder_input_data = np.zeros(
    (len(input_seqs), max_length_in, len(one_hot_input)),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')

for i, (input_seqs, output_seqs) in enumerate(zip(input_seqs, output_seqs)):
    for t, char in enumerate(input_seqs):
        #print(t)
        encoder_input_data[i, t, one_hot_input[char]] = 1.
    for t, char in enumerate(output_seqs):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, one_hot_input[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, one_hot_input[char]] = 1

In [11]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)

(77510, 351, 5)
(77510, 351, 5)


In [12]:
#one hot encode val data

#ENCODE
encoder_input_data_val = np.zeros(
    (len(val_input_seqs), max_length_in, len(one_hot_input)),
    dtype='float32')
decoder_input_data_val = np.zeros(
    (len(val_input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')
decoder_target_data_val = np.zeros(
    (len(val_input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')

for i, (val_input_seqs, val_output_seqs) in enumerate(zip(val_input_seqs, val_output_seqs)):
    for t, char in enumerate(val_input_seqs):
        #print(t)
        encoder_input_data_val[i, t, one_hot_input[char]] = 1.
    for t, char in enumerate(val_output_seqs):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data_val[i, t, one_hot_input[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data_val[i, t - 1, one_hot_input[char]] = 1

In [13]:
print(encoder_input_data_val.shape)
print(decoder_input_data_val.shape)

(12269, 351, 5)
(12269, 351, 5)


In [14]:
#one hot encode test data

#ENCODE
encoder_input_data_test = np.zeros(
    (len(test_input_seqs), max_length_in, len(one_hot_input)),
    dtype='float32')
decoder_input_data_test = np.zeros(
    (len(test_input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')
decoder_target_data_test = np.zeros(
    (len(test_input_seqs), max_length_out, len(one_hot_input)),
    dtype='float32')

for i, (test_input_seqs, test_output_seqs) in enumerate(zip(test_input_seqs, test_output_seqs)):
    for t, char in enumerate(test_input_seqs):
        #print(t)
        encoder_input_data_test[i, t, one_hot_input[char]] = 1.
    for t, char in enumerate(test_output_seqs):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data_test[i, t, one_hot_input[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data_test[i, t - 1, one_hot_input[char]] = 1

In [15]:
print(encoder_input_data_test.shape)
print(decoder_input_data_test.shape)

(12107, 351, 5)
(12107, 351, 5)


In [16]:
#parameters to change
num_encoder_tokens = len(one_hot_input)
num_decoder_tokens = len(one_hot_input)
latent_dim = 100

In [17]:
#model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
e_lstm_1 = Bidirectional(LSTM(latent_dim, return_sequences = True))(encoder_inputs)
e_dropout = Dropout(0.5)(e_lstm_1)
e_lstm_2, fh2, fc2, bh2, bc2 = Bidirectional(LSTM(latent_dim, return_sequences = True, return_state=True))(e_dropout)
#e_lstm_3 = Bidirectional(LSTM(latent_dim, return_sequences = True))
h2=Concatenate()([fh2,bh2])
c2=Concatenate()([fc2,bc2])

timed_output = TimeDistributed(Dense(num_decoder_tokens, activation = "softmax"))(e_lstm_2)
noise_class = Dense(1, activation = "tanh")(Concatenate()([h2, c2]))
#noise_class = Activation("tanh")(noise_class)
#print(noise_class)

In [20]:
print(timed_output)

Tensor("time_distributed_1/Reshape_1:0", shape=(?, ?, 5), dtype=float32)


In [21]:
model= Model(encoder_inputs, outputs=[timed_output,noise_class])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 5)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, None, 200)    84800       input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 200)    0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) [(None, None, 200),  240800      dropout_1[0][0]                  
__________________________________________________________________________________________________
concatenat

In [22]:
adam = keras.optimizers.Adam(lr = .001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
weight_nucleotide=1.
weight_noise= 0.2
model.compile(optimizer= adam, loss=['categorical_crossentropy','binary_crossentropy'], metrics = ['accuracy'], loss_weights=[weight_nucleotide, weight_noise])


In [28]:
#NOTE: for the cell below
#1. Modelcheckpoint doesn't take multiple arguments as a list to 'monitor', since what it does is basically check whethter the
# quantity to monitor has improved over the last epoch and save the model
#2.Specify the val accuracy corresponding to the output you care
#3. file path also should have the same val_accuracy
#4. Working on performing something like having a single loss function (as an average or something) instead of two loss functions 
# right now. Then we'll need only one val_accuracy. Right now don't think avg loss is a better idea


In [25]:
history = History()
filepath="seqWeights/MultiLossLSTM-test-{epoch:02d}-{val_time_distributed_1_acc:.2f}.hdf5"
monitor_param='val_time_distributed_1_acc'
checkpoint = ModelCheckpoint(filepath, monitor=monitor_param, verbose=1, save_best_only=True, mode='max')

numExamples = 200
batch_size = 100
epochs = 2
model.fit(encoder_input_data[0:numExamples, :, :],
          [decoder_input_data[0:numExamples, :, :],np.array(labels_train)[0:numExamples]],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(encoder_input_data_val[0:numExamples,:,:],[decoder_input_data_val[0:numExamples, :, :],np.array(labels_val)[0:numExamples]]), verbose = 1,
          callbacks = [history, checkpoint])

Train on 200 samples, validate on 200 samples
Epoch 1/2
200/200 [==============================] - 14s 71ms/step - loss: 1.1313 - time_distributed_1_loss: 1.0563 - dense_2_loss: 0.3753 - time_distributed_1_acc: 0.2594 - dense_2_acc: 1.0000 - val_loss: 1.0436 - val_time_distributed_1_loss: 1.0183 - val_dense_2_loss: 0.1269 - val_time_distributed_1_acc: 0.2528 - val_dense_2_acc: 1.0000

Epoch 00001: val_time_distributed_1_acc improved from -inf to 0.25281, saving model to seqWeights/MultiLossLSTM-test-01-0.25.hdf5
Epoch 2/2
200/200 [==============================] - 14s 72ms/step - loss: 1.0222 - time_distributed_1_loss: 1.0052 - dense_2_loss: 0.0849 - time_distributed_1_acc: 0.2574 - dense_2_acc: 1.0000 - val_loss: 0.9949 - val_time_distributed_1_loss: 0.9929 - val_dense_2_loss: 0.0099 - val_time_distributed_1_acc: 0.2527 - val_dense_2_acc: 1.0000

Epoch 00002: val_time_distributed_1_acc did not improve from 0.25281


In [ ]:
score= model.evaluate(encoder_input_data_test[0:numExamples,:,:], np.array(labels_test)[0:numExamples], verbose=0)
print("scores")
print(score)

In [26]:
print(model.predict(encoder_input_data_test[0:2, :, :]))
print(np.array(labels_test)[0:2])
#need to work on this

[array([[[0.22730513, 0.22640508, 0.19525397, 0.25066787, 0.10036795],
        [0.22786762, 0.21644306, 0.19795269, 0.2677272 , 0.0900095 ],
        [0.22909367, 0.20407104, 0.1948256 , 0.29134625, 0.08066343],
        ...,
        [0.20849599, 0.19009958, 0.2028314 , 0.22064136, 0.17793164],
        [0.2079552 , 0.18991601, 0.20310672, 0.21937504, 0.17964701],
        [0.20739338, 0.18975265, 0.20344767, 0.21792847, 0.18147786]],

       [[0.2274647 , 0.22510116, 0.1947394 , 0.24727607, 0.10541873],
        [0.22870563, 0.21772306, 0.19721518, 0.26146945, 0.09488663],
        [0.23089944, 0.20865302, 0.19431643, 0.28084564, 0.08528544],
        ...,
        [0.20849599, 0.19009955, 0.20283139, 0.22064136, 0.17793162],
        [0.2079552 , 0.18991598, 0.2031067 , 0.21937504, 0.179647  ],
        [0.20739336, 0.18975264, 0.20344765, 0.21792847, 0.18147784]]],
      dtype=float32), array([[0.99162036],
       [0.9874339 ]], dtype=float32)]
[[1]
 [1]]
